# OpenSea V1


In [12]:
import pandas as pd

from credmark.cmf.ipython import create_cmf
from credmark.cmf.types import Token, Contract, Address, Account, BlockNumber, Records

# fix_block_number = None
fix_block_number = 17527032

cmf_param = {
    'block_number': fix_block_number,
    'chain_id': 1,
    'use_local_models': '*',
}
context, _model_loader = create_cmf(cmf_param, True)
fix_block_number = context.block_number
print(fix_block_number)

Credmark context created with 
- chain_id=1
- block_number=17527032
- chain_to_provider_url=http://192...8545
- model_loader_path=['/home/yangye/dev/credmark/credmark-models-py/models']
- api_url=https://gateway.credmark.com
- use_local_models=*

17527032


https://github.com/ProjectOpenSea/seaport

- Contract Canonical Cross-chain Deployment Address
- Seaport 1.1 0x00000000006c3852cbEf3e08E8dF289169EdE581
- Seaport 1.2 0x00000000000006c7676171937C444f6BDe3D6282
- Seaport 1.3 0x0000000000000aD24e80fd803C6ac37206a45f15
- Seaport 1.4 0x00000000000001ad428e4906aE43D8F9852d0dD6
- Seaport 1.5 0x00000000000000ADc04C56Bf30aC9d3c0aAF14dC


In [4]:
opensea_v1 = Contract('0x7Be8076f4EA4A4AD08075C2508e481d6C946D12b')

opensea_v1_blk = context.run_model('token.deployment', opensea_v1)[  # type: ignore
    'deployed_block_number']
opensea_v1_blk

5774644

In [30]:
df = pd.DataFrame(opensea_v1.fetch_events(
    opensea_v1.events.OrdersMatched,
    from_block=5774644,
    to_block=5774644+10000,
    contract_address=opensea_v1.address))

if not df.empty:
    df = (
        df.loc[:, ['blockNumber', 'logIndex', 'maker', 'taker',
                   'metadata', 'buyHash', 'sellHash', 'price', 'transactionHash']]
        .assign(metadata=lambda x: x.metadata.apply(lambda y: y.hex()),
                buyHash=lambda x: x.buyHash.apply(lambda y: y.hex()),
                sellHash=lambda x: x.sellHash.apply(lambda y: y.hex()),
                transactionHash=lambda x: x.transactionHash.apply(lambda y: y.hex()),
                price=lambda x: x.price / 1e18,)
    )

    display(df.loc[0].T)

    display(df.loc[0].metadata)

    display(len(df.loc[0].transactionHash),
            len(df.loc[0].metadata),
            len(df.loc[0].buyHash),
            len(df.loc[0].sellHash))

# df.loc[:, 'txHash'] = df.transactionHash.apply(lambda x: x.hex())
# df.query('txHash == "0x7efdbef1c377126b5bb5381b8f46a5dd7cb4b992b5f2f48a5f47eec86297c281"').T

blockNumber                                                  5779378
logIndex                                                          55
maker                     0x6Be4a7bbb812Bfa6A63126EE7b76C8A13529BDB8
taker                     0x0239769A1aDF4DeF9f07Da824B80B9C4fCB59593
metadata           0000000000000000000000000000000000000000000000...
buyHash            c1f12102332b8d0dfe98a6e2045860dfd22a30388c97b4...
sellHash           0000000000000000000000000000000000000000000000...
price                                                          0.001
transactionHash    0xb066452c1e87696392c9e74fd0159dea8b53d2162138...
Name: 0, dtype: object

'0000000000000000000000000000000000000000000000000000000000000000'

66

64

64

64

In [ ]:
df = pd.DataFrame(opensea_v1.fetch_events(
    opensea_v1.events.Orders,
    from_block=14029202,
    to_block=14029202,
    contract_address=opensea_v1.address))

## V2 Seaport


In [10]:
from models.credmark.tokens.nft.nft_transfer import SeaportSpentItems, SeaportReceivedItems

seaport = Contract('0x00000000000000ADc04C56Bf30aC9d3c0aAF14dC')
df_seaport = pd.DataFrame(seaport.fetch_events(seaport.events.OrderFulfilled, from_block=17512390, to_block=17512390))

display(df_seaport.loc[0].T)

if not df_seaport.empty:
    df_seaport = (
        df_seaport.loc[:, ['blockNumber', 'logIndex', 'address', 'transactionHash',
                           'offerer', 'zone',
                           'orderHash', 'recipient', 'offer', 'consideration']]
        .assign(transactionHash=lambda x: x.transactionHash.apply(lambda y: y.hex()),
                orderHash=lambda x: x.orderHash.apply(lambda y: y.hex(),),
                offer=lambda x: x.offer.apply(lambda x: SeaportSpentItems.from_list(x).json()),
                consideration=lambda x: x.consideration.apply(lambda x: SeaportReceivedItems.from_list(x).json())
                ))

    # display(SeaportSpentItem.from_tuple(df_seaport.loc[0, 'offer'][0]))

    display(df_seaport.T)

    display(df_seaport.dtypes)

    display(len(df_seaport.loc[0].transactionHash),
            len(df_seaport.loc[0].offerer),
            len(df_seaport.loc[0].zone),
            len(df_seaport.loc[0].orderHash),
            len(df_seaport.loc[0].recipient))


# df_seaport.offer[0]

args                (offerer, zone, orderHash, recipient, offer, c...
event                                                  OrderFulfilled
logIndex                                                          262
transactionIndex                                                  128
transactionHash     b'M\x0b\xfd\xf2[Ff\xb7w\xfe\xd8\x89\x85\\\x01]...
address                    0x00000000000000ADc04C56Bf30aC9d3c0aAF14dC
blockHash           b"\x98d\xdc\xeel#\x82t\xd9\x91\x9d\xe2'.\xa7[\...
blockNumber                                                  17512390
offerer                    0x7c4b23fD000D6FeD7B3091c5b24c60922E733C51
zone                       0x004C00500000aD104D7DBd00e3ae0A5C00560C00
orderHash           b's\xe26\xd9\xbe\x18[\xa8\x9e\x13nv2\x9c\xd6\x...
recipient                  0x372bAcbC17B48EE6D532750f5A46bDa18b050a6e
offer               [(2, 0x1A2A5Cdb346b8C5C83764C27266C5E52B35f93B...
consideration       [(0, 0x000000000000000000000000000000000000000...
Name: 0, dtype: obje

,0,1,2
blockNumber,17512390,17512390,17512390
logIndex,262,264,319
address,0x00000000000000ADc04C56Bf30aC9d3c0aAF14dC,0x00000000000000ADc04C56Bf30aC9d3c0aAF14dC,0x00000000000000ADc04C56Bf30aC9d3c0aAF14dC
transactionHash,0x4d0bfdf25b4666b777fed889855c015d141d55de8e29...,0x00a234646f6381d699198e3a6429966429e4245c7b5a...,0x0089c3509e5adc6d54979ba08a3278f59a272531e503...
offerer,0x7c4b23fD000D6FeD7B3091c5b24c60922E733C51,0xA60219f09e24762CA352109C7c19F3cBAdD2D62e,0xdbA15633594ec70A0B16365953340a5bbd82D3F2
zone,0x004C00500000aD104D7DBd00e3ae0A5C00560C00,0x004C00500000aD104D7DBd00e3ae0A5C00560C00,0x004C00500000aD104D7DBd00e3ae0A5C00560C00
orderHash,73e236d9be185ba89e136e76329cd6a220ab63bd871aac...,e360784dfb22f34a07949b17dc9ab3b2dee0db82a78fc2...,619ccd9d6b56b52cbe353509aca5d935d2040506960b7b...
recipient,0x372bAcbC17B48EE6D532750f5A46bDa18b050a6e,0xf85139701FcCa36b2169610A376cd6276901239A,0xf901905D61D4927b029160527bA5c1C88C277373
offer,"{""items"": [{""itemType"": 2, ""token"": ""0x1a2a5cd...","{""items"": [{""itemType"": 2, ""token"": ""0x54d0352...","{""items"": [{""itemType"": 2, ""token"": ""0x4d7d2e2..."
consideration,"{""items"": [{""itemType"": 0, ""token"": ""0x0000000...","{""items"": [{""itemType"": 0, ""token"": ""0x0000000...","{""items"": [{""itemType"": 0, ""token"": ""0x0000000..."


blockNumber         int64
logIndex            int64
address            object
transactionHash    object
offerer            object
zone               object
orderHash          object
recipient          object
offer              object
consideration      object
dtype: object

66

42

42

64

42